In [1]:
!pip install transformers datasets
!pip uninstall accelerate transformers
!pip install accelerate transformers[torch]

Found existing installation: accelerate 0.27.2
Uninstalling accelerate-0.27.2:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/lib/python3.10/dist-packages/accelerate-0.27.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? y
  Successfully uninstalled accelerate-0.27.2
Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Would remove:
    /usr/local/bin/transformers-cli
    /usr/local/lib/python3.10/dist-packages/transformers-4.35.2.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
y
y
  Successfully uninstalled transformers-4.35.2
  Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
data=load_dataset('conll2003')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
data


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [4]:
data['train']


Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [5]:
data['train'].features['tokens']


Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)

In [6]:
data['train'].features['ner_tags']


Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [7]:
data['train'].features['ner_tags'].feature.names


['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [8]:

label_names=data['train'].features['ner_tags'].feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:

from transformers import AutoTokenizer
checkpoint='distilbert-base-cased'
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [10]:
idx=0
t=tokenizer(data['train'][idx]['tokens'],is_split_into_words=True)
t

{'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:


type(t)
t.tokens

<bound method BatchEncoding.tokens of {'input_ids': [101, 7270, 22961, 1528, 1840, 1106, 21423, 1418, 2495, 12913, 119, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}>

In [12]:
begin2inside={
    1:2,
    3:4,
    5:6,
    7:8

}
def align_targets(labels,word_ids):
  aligned_labels=[]
  last_word=None
  for word in word_ids:

    if word is None:
      label=-100
    elif word!=last_word:
      label=labels[word]
    else:
      label=labels[word]
      if label in begin2inside:
        label=begin2inside[label]
    aligned_labels.append(label)
    last_word=word
  return aligned_labels

In [13]:

#try our functin
labels=data['train'][idx]['ner_tags']
labels

[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [14]:

word_ids=t.word_ids()
word_ids

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [15]:
aligned_targets=align_targets(labels,word_ids)
print(aligned_targets)

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


In [16]:

aligned_labels=[label_names[t] if t>=0 else  None for t in aligned_targets ]
aligned_labels

[None, 'B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O', 'O', None]

In [17]:
for x,y in zip(t.tokens(),aligned_labels):
  print(f'{x}\t{y}')

[CLS]	None
EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
la	O
##mb	O
.	O
[SEP]	None


In [18]:
#make up fake input just to test it
words=[
    '[CLS]','Ger','man','call','to','boycott','micro','soft','[SEP]']
word_ids=[None,0,0,1,2,3,4,4,None]
labels=[7,0,0,0,3]
aligned_targets=align_targets(labels,word_ids)
aligned_targets

[-100, 7, 8, 0, 0, 0, 3, 4, -100]

In [19]:
aligned_labels=[label_names[t] if t>=0 else None for t in aligned_targets]
aligned_labels

[None, 'B-MISC', 'I-MISC', 'O', 'O', 'O', 'B-ORG', 'I-ORG', None]

In [20]:
for x,y in zip(words,aligned_labels):
  print(f'{x}\t{y}')

[CLS]	None
Ger	B-MISC
man	I-MISC
call	O
to	O
boycott	O
micro	B-ORG
soft	I-ORG
[SEP]	None


In [21]:
def tokenize_fn(batch):
  tokenized_inputs=tokenizer(
      batch['tokens'],
      truncation=True,
      is_split_into_words=True
  )

  labels_batch=batch['ner_tags']#original tags
  aligned_labels_batch=[]

  for i,labels in enumerate(labels_batch):
    word_ids=tokenized_inputs.word_ids(i)
    aligned_labels_batch.append(align_targets(labels,word_ids))

  #recall the target must be stored in any key called labels
  tokenized_inputs['labels']=aligned_labels_batch
  return tokenized_inputs

In [22]:
data['train'].column_names

['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags']

In [23]:
tokenized_datasets=data.map(tokenize_fn, batched=True, remove_columns=data['train'].column_names,)
tokenized_datasets

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [24]:
len(tokenized_datasets['train'][0].get('input_ids'))

12

In [25]:
len(tokenized_datasets['train'][0].get('labels'))

12

In [26]:
from transformers import DataCollatorForTokenClassification
data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)

In [27]:
tokenized_datasets['train'][0]

{'input_ids': [101,
  7270,
  22961,
  1528,
  1840,
  1106,
  21423,
  1418,
  2495,
  12913,
  119,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]}

In [28]:
tokenized_datasets['train'][0:2]

{'input_ids': [[101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  [101, 1943, 14428, 102]],
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]],
 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100], [-100, 1, 2, -100]]}

In [29]:
[tokenized_datasets['train'][i] for i in range(2)]

[{'input_ids': [101,
   7270,
   22961,
   1528,
   1840,
   1106,
   21423,
   1418,
   2495,
   12913,
   119,
   102],
  'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
  'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]},
 {'input_ids': [101, 1943, 14428, 102],
  'attention_mask': [1, 1, 1, 1],
  'labels': [-100, 1, 2, -100]}]

In [30]:
#example
batch=data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch['labels']

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

In [31]:
!pip install seqeval

In [32]:
from datasets import load_metric
metric=load_metric('seqeval')

<ipython-input-32-3bdac680cea9>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=load_metric('seqeval')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:753: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [33]:
metric.compute(predictions=[[0,0,0]],references=[[0,0,1]])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:5

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [34]:
metric.compute(predictions=[['A','A','A']],references=[['A','B','A']])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: A seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 0.6666666666666666}

In [42]:
import numpy as np
def compute_metrics(logits_and_labels):
  logits,labels=logits_and_labels
  preds=np.argmax(logits,axis=-1)

  #remove -100 from labels and predictions
  #and convert the label_ids to label_names
  str_labels=[
      [label_names[t] for t in label if t!= -100] for label in labels
  ]
  #do the same for predictions whenever true label is -100
  str_preds=[
      [label_names[p] for p, t in zip(pred,targ) if t!= -100] for pred,targ in zip(preds,labels)
  ]
  the_metrics=metric.compute(predictions=str_preds,references=str_labels)
  return{
      'precision':the_metrics['overall_precision'],
      'recall':the_metrics['overall_recall'],
      'f1':the_metrics['overall_f1'],
      'accuracy':the_metrics['overall_accuracy']
  }


In [36]:
id2label={k:v for k ,v in enumerate(label_names)}
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [37]:
label2id={v:k for k ,v in id2label.items()}
label2id

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [38]:
from transformers import AutoModelForTokenClassification
model=AutoModelForTokenClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
from transformers import TrainingArguments

In [40]:
training_arguments=TrainingArguments(
    'distilbert-finetuned-ner',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=5,
    weight_decay=0.01
)

In [43]:
from transformers import Trainer
trainer=Trainer(
    model=model,
    args=training_arguments,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer

)
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.038000,0.087948,0.904247,0.928139,0.916037,0.980470
2,0.033400,0.068412,0.910875,0.937395,0.923945,0.983826
3,0.020800,0.076199,0.916543,0.938909,0.927592,0.983796
4,0.009400,0.084556,0.924435,0.942949,0.933600,0.984827
5,0.004500,0.087294,0.925457,0.946483,0.935852,0.985253


TrainOutput(global_step=8780, training_loss=0.022691837693129695, metrics={'train_runtime': 532.4008, 'train_samples_per_second': 131.865, 'train_steps_per_second': 16.491, 'total_flos': 768370385878326.0, 'train_loss': 0.022691837693129695, 'epoch': 5.0})

In [45]:
trainer.save_model('my_ner_saved_model')


In [46]:
from transformers import pipeline
ner=pipeline(
    'token-classification',
    model='my_ner_saved_model',
    aggregation_strategy='simple',
    device=0
)
s='Hari is the hero of movie Bahubali which was shooted in USA'
ner(s)

[{'entity_group': 'PER',
  'score': 0.99972314,
  'word': 'Hari',
  'start': 0,
  'end': 4},
 {'entity_group': 'PER',
  'score': 0.89911705,
  'word': 'Bahubali',
  'start': 26,
  'end': 34},
 {'entity_group': 'LOC',
  'score': 0.999479,
  'word': 'USA',
  'start': 56,
  'end': 59}]

In [48]:
s='USA is the country whose president is Joe Biden '
ner(s)

[{'entity_group': 'LOC',
  'score': 0.9999007,
  'word': 'USA',
  'start': 0,
  'end': 3},
 {'entity_group': 'PER',
  'score': 0.999821,
  'word': 'Joe Biden',
  'start': 38,
  'end': 47}]

In [ ]:
#https://drive.google.com/drive/folders/1Vr5LbgoffLbQ0VaKf6QMIMOR2VpXtK9R